Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Explain regression model predictions using MimicExplainer
_**This notebook showcases how to use the interpret-community repo to help interpret and inspect replication metrics for predictions given by the MimicExplainer's surrogate model for a regression model.**_


## Table of Contents

1. [Introduction](#Introduction)
1. [Project](#Project)
1. [Setup](#Setup)
1. [Run model explainer locally at training time](#Explain)
    1. Load the housing house price data
    1. Train a GradientBoosting regression model, which you want to explain
    1. Initialize the `MimicExplainer`
    1. Inspect replication metrics to see how close is the global surrogate model to the trained model
    1. Try different surrogate models to see if the replication measure changes
    1. Generate global explanations
    1. Generate local explanations
1. [Next steps](#Next)

<a id='Introduction'></a>
## 1. Introduction

This notebook illustrates how to use interpret-community to help interpret regression model predictions at training time using `MimicExplainer`. It demonstrates how to determine which global surrogate model better replicates the trained model by using metrics like `r2_score`. It also demonstrates the API calls needed to obtain the global and local interpretations.

Three tabular data explainers are demonstrated: 
- MimicExplainer (global surrogate)

| ![Interpretability Toolkit Architecture](./img/interpretability-architecture.png) |
|:--:|
| *Interpretability Toolkit Architecture* |

<a id='Project'></a>       
## 2. Project

The goal of this project is to predict California Housing Prices by using scikit-learn and locally running the model explainer:

1. Train a GradientBoosting regression model using Scikit-learn
2. Use `get_surrogate_model_replication_measure()` to determine how close are the predictions of the global surrogate model are to the regression model trained in step 1.
3. Run `explain_model()` globally and locally with full dataset in local mode, which doesn't contact any Azure services.

<a id='Setup'></a>
## 3. Setup

If you are using Jupyter notebooks, the extensions should be installed automatically with the package.
If you are using Jupyter Labs run the following command:
```
(myenv) $ jupyter labextension install @jupyter-widgets/jupyterlab-manager
```


<a id='Explain'></a>
## 4. Run model explainer locally at training time

In [ ]:
from sklearn import datasets
from sklearn.ensemble import GradientBoostingRegressor

from interpret.ext.blackbox import MimicExplainer
# You can use one of the following four interpretable models as a global surrogate to the black box model
from interpret.ext.glassbox import LGBMExplainableModel
from interpret.ext.glassbox import LinearExplainableModel
from interpret.ext.glassbox import SGDExplainableModel
from interpret.ext.glassbox import DecisionTreeExplainableModel


### Load the housing house price data

In [ ]:
housing_data = datasets.fetch_california_housing()

In [ ]:
# Split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(housing_data.data, housing_data.target, test_size=0.2, random_state=0)

### Train a GradientBoosting regression model, which you want to explain

In [ ]:
reg = GradientBoostingRegressor(n_estimators=100, max_depth=4,
                                learning_rate=0.1, loss='huber',
                                random_state=1)
model = reg.fit(x_train, y_train)

### Initialize the `MimicExplainer`

In [ ]:
# augment_data is optional and if true, oversamples the initialization examples to improve surrogate model accuracy to fit original model.  Useful for high-dimensional data where the number of rows is less than the number of columns. 
# max_num_of_augmentations is optional and defines max number of times we can increase the input data size.
# LGBMExplainableModel can be replaced with LinearExplainableModel, SGDExplainableModel, or DecisionTreeExplainableModel
explainer = MimicExplainer(model, 
                           x_train, 
                           LGBMExplainableModel, 
                           augment_data=True, 
                           max_num_of_augmentations=10, 
                           features=housing_data.feature_names)

### Inspect replication metrics to see how close is the global surrogate model to the trained model
For regression model the function `get_surrogate_model_replication_measure()` will return the `r2_score` between the predictions of the trained model and the predictions of the global surrogate model.

In [ ]:
explainer.get_surrogate_model_replication_measure(training_data=x_train)

### Try different surrogate models to see if the replication measure changes
Let's choose a different global surrogate (out of `LinearExplainableModel`, `SGDExplainableModel`, or `DecisionTreeExplainableModel`) and see how the replication metrics change

In [ ]:
explainer = MimicExplainer(model, 
                           x_train, 
                           LinearExplainableModel, 
                           augment_data=True, 
                           max_num_of_augmentations=10, 
                           features=housing_data.feature_names)
                
explainer.get_surrogate_model_replication_measure(training_data=x_train)

### Generate global explanations
Explain overall model predictions (global explanation)

In [ ]:
# Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
# x_train can be passed as well, but with more examples explanations will take longer although they may be more accurate
global_explanation = explainer.explain_global(x_test)

# Note: if you used the PFIExplainer in the previous step, use the next line of code instead
# global_explanation = explainer.explain_global(x_test, true_labels=y_test)

In [ ]:
# Sorted SHAP values 
print('ranked global importance values: {}'.format(global_explanation.get_ranked_global_values()))
# Corresponding feature names
print('ranked global importance names: {}'.format(global_explanation.get_ranked_global_names()))
# Feature ranks (based on original order of features)
print('global importance rank: {}'.format(global_explanation.global_importance_rank))

In [ ]:
# Print out a dictionary that holds the sorted feature importance names and values
print('global importance rank: {}'.format(global_explanation.get_feature_importance_dict()))

#### Explain overall model predictions as a collection of local (instance-level) explanations

In [ ]:
# feature shap values for all features and all data points in the training data
print('local importance values: {}'.format(global_explanation.local_importance_values))

### Generate local explanations
Explain local data points (individual instances)

In [ ]:
# E.g., Explain the first data point in the test set
local_explanation = explainer.explain_local(x_test[0,:])

In [ ]:
# Sorted local feature importance information; reflects the original feature order
sorted_local_importance_names = local_explanation.get_ranked_local_names()
sorted_local_importance_values = local_explanation.get_ranked_local_values()

print('sorted local importance names: {}'.format(sorted_local_importance_names))
print('sorted local importance values: {}'.format(sorted_local_importance_values))

<a id='Next'></a>
## 5. Next steps
Learn about other use cases of the interpret-community package:
       
1. [Training time: binary classification problem](./explain-binary-classification-local.ipynb)
1. [Training time: multiclass classification problem](./explain-multiclass-classification-local.ipynb)
1. Explain models with engineered features:
    1. [Simple feature transformations](./simple-feature-transformations-explain-local.ipynb)
    1. [Advanced feature transformations](./advanced-feature-transformations-explain-local.ipynb)